In [ ]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

In [ ]:
import numpy as np 

y = np.array([0.5, 1.0, 0.75, 0.60, 0.66])

average_y = np.mean(y)

# Calculate the MSE for each value in y with respect to the average
mse_scores = (y - average_y) ** 2

mse_scores, average_y

In [ ]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

In [ ]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
sbml_dirpath = '../biosimulator_processes/model_files/sbml'
model_fp = os.path.join(sbml_dirpath, f'{biomodel_id}_url.xml')
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'tellurium']

In [ ]:
def run_workflow(bio_id, duration):
    from tempfile import mkdtemp
    from biosimulator_processes.io import fetch_sbml_file
    directory = mkdtemp()
    model_fp = fetch_sbml_file(bio_id, save_dir=directory)
    COMPOSITE_DOC = {
        'copasi_simple': {
            '_type': 'process',
              'address': 'local:copasi',
              'config': {'model': {'model_source': model_fp}},
              'inputs': {'floating_species_concentrations': ['copasi_simple_floating_species_concentrations_store'],
               'model_parameters': ['model_parameters_store'],
               'time': ['time_store'],
               'reactions': ['reactions_store']},
              'outputs': {'floating_species_concentrations': ['copasi_simple_floating_species_concentrations_store'],
               'time': ['time_store']}
        },
        'amici_simple': {
            '_type': 'process',
            'address': 'local:amici',
            'config': {'model': {'model_source': model_fp}},
            'inputs': {
                'floating_species_concentrations': ['amici_simple_floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']},
            'outputs': {
                'floating_species_concentrations': ['amici_simple_floating_species_concentrations_store'],
                'time': ['time_store']}
        },
        'emitter': {
             '_type': 'step',
              'address': 'local:ram-emitter',
              'config': {
                  'emit': {
                      'copasi_simple_floating_species_concentrations': 'tree[float]',
                      'amici_simple_floating_species_concentrations': 'tree[float]',
                      'tellurium_simple_floating_species_concentrations': 'tree[float]',
                      'time': 'float'
                  }
              },
              'inputs': {
                  'copasi_simple_floating_species_concentrations': ['copasi_simple_floating_species_concentrations_store'],
                  'amici_simple_floating_species_concentrations': ['amici_simple_floating_species_concentrations_store'],
                  'tellurium_simple_floating_species_concentrations': ['tellurium_simple_floating_species_concentrations_store'],
                  'time': ['time_store']
              }
        },
        'tellurium_simple': {
            '_type': 'process',
              'address': 'local:tellurium',
              'config': {'model': {'model_source': model_fp}},
              'inputs': {'floating_species_concentrations': ['tellurium_simple_floating_species_concentrations_store'],
               'model_parameters': ['model_parameters_store'],
               'time': ['time_store'],
               'reactions': ['reactions_store']},
              'outputs': {'floating_species_concentrations': ['tellurium_simple_floating_species_concentrations_store'],
               'time': ['time_store']}}}

    comp = Composite(config={'state': COMPOSITE_DOC}, core=CORE)
    comp.run(duration)
    print(model_fp)
    return comp.gather_results()

In [ ]:
data = run_workflow('BIOMD0000000744', duration)

In [ ]:
compare = {
    'compare_ode': {
            '_type': 'step',
              'address': 'local:compare_ode_step',
              'config': {'biomodel_id': biomodel_id, 'duration': duration},
              'inputs': {},
              'outputs': {
                'comparison_data': ['comparison_store']
            }
    },
    'emitter': {
             '_type': 'step',
              'address': 'local:ram-emitter',
              'config': {
                  'emit': {
                      'comparison_data': 'tree[any]'
                  }
              },
              'inputs': {
                  'comparison_data': ['comparison_store']
              }
    }
}


wf = Composite(config={'state': compare}, core=CORE)
wf.run(1)
comparison_results = wf.gather_results()


pp(comparison_results)